# Cloud Computing Assignment 1 - Team B6

# Part 1

In [ ]:
import requests
import pandas as pd
import hashlib
import json

In [ ]:
base_url = "http://gateway.marvel.com/v1/public/"
publicKey = "909856c3e3c3bc72bedd8567fdd85064"
privateKey = "8e37626d552a3bd776adf5bd1d5e4b372386d208"
ts = '1'
hashKey = hashlib.md5((ts+privateKey+publicKey).encode('utf-8')).hexdigest()

params = { 'apikey' : publicKey,'ts': ts,'hash': hashKey, 'limit':30}

In [ ]:
response = requests.get(base_url+'characters',params).json()
data = response['data']['results']

In [ ]:
elements = len(data)
i=0
characters = []

while i < elements:
    characters.append(data[i]['name'])
    i+=1
    
print(characters)
print('Type of character records:',type(characters[0]))   

In [ ]:
i=0
ids = []

while i < elements:
    ids.append(data[i]['id'])
    i+=1


ids_str = [str(x) for x in ids]    
print(ids_str)
print('Type of ID records:',type(ids_str[0]))

In [ ]:
i=0
comics = []
total = 0

while i < elements:
    if data[i]['comics']['available']>0:
        comics.append(data[i]['comics']['available'])
        total = total + data[i]['comics']['available']
    else: 
        comics.append(None)
    i+=1

print('Total comics for all characters:', total)
#print(comics)
print('Type of comic records:',type(comics[0]))

In [ ]:
i=0
series = []
total = 0

while i < elements:
    if data[i]['series']['available']>0:
        series.append(data[i]['series']['available'])
        total = total + data[i]['series']['available']
    else:
        series.append(None)
    i+=1

print('Total series for al characters:',total)
#print(series)
print('Type of series records:',type(series[0]))

In [ ]:
i=0
events = []
total = 0

while i < elements:
    if data[i]['events']['available'] > 0:
        events.append(data[i]['events']['available'])
        total = total + data[i]['events']['available']
    else:
        events.append(None)
    i+=1

print('Total events for al characters:',total)    
#print(events)
print('Type of events records:',type(events[0]))

In [ ]:
i=0
max_price = []

while i < elements:
    
    response2 = requests.get(base_url+'characters/'+str(ids[i])+'/comics',params).json()
    data_prices = response2['data']['results']
    range_prices = len(data_prices)

    
    j = 0
    aux_price=[]
    
    while j < range_prices:
        
        if len(data_prices[j]['prices']) == 2:
                aux_price.append(max(data_prices[j]['prices'][0]['price'], data_prices[j]['prices'][1]['price']))
        else:
                aux_price.append(data_prices[j]['prices'][0]['price'])
                      
        j+=1

    if len(aux_price)>0:
        max_price.append(max(aux_price))
    else:
        max_price.append(None)
   
    i+=1
               
        
        
print(max_price)
#print(len(max_price))

In [ ]:
df = {'Character Name':  characters,
     'Character ID': ids,
     'Total Available Events':events,
     'Total Available Series':series,
     'Total Available Comics':comics,
     'Price of the Most Expensive Comic':max_price}



result = pd.DataFrame(df)
#print(result)

In [ ]:
result.to_csv('data.csv')

# Part 2

In this API you can request individual marvel characters from the data.csv file. Likewise, you can store users and passwords in the user.csv file.

First, run the terminal from within the jupyter notebook. Once in the terminal, type "python api2.py", this will start the api.

Make sure you have the following libraries installed (pip install flask_jwt_extended,pip install flask-bcrypt,pip install flask,pip install flask__restful) or the API will not start.

If the API has started succesfully you will see the following code at the end of the terminal: Running on "address"

**1. To use dataframes and API requests we must import the two libraries shown below**

In [ ]:
import requests
import pandas as pd

**2. To retrieve the whole DataFrame in json format we must use the following notation.**

**If the address below doesn't work, substitute it with 'http://localhost:5000/characters'.**

In [ ]:
# Retrieve the whole DataFrame in json format
requests.get('http://127.0.0.1:5000/characters').json()


**3. With the following request, we can retrieve information for a single entry or for a list of entries identified by Character ID. This can be done by changing the value for the Character ID in the params dictionary**

In [ ]:
#Retrieve information for a single entry or for a list of entries identified by Character ID
requests.get('http://127.0.0.1:5000/characters', params={'Character ID':1009146}).json()

ids = [1009146,1011120]
requests.get('http://127.0.0.1:5000/characters', params={'Character ID':ids}).json()

**4. With the following request, we can retrieve information for a single entry or for a list of entries identified by Character Name.This can be done by changing the value for the Character Name in the params dictionary**

In [ ]:
#Retrieve information for a single entry or for a list of entries identified by Character Name

names = ['3-D Man','Albion']

requests.get('http://127.0.0.1:5000/characters', params={'Character Name':names}).json()

**5. This section enables the user to add a new character to the existing DataFrame by specifying its characteristics, alter the values for the distinict keys in the params dictionary, by using your authorization token.**

**Note that you don't have to insert your access token given its already saved, since it was saved in part 10.**

**Duplicates will not work**

In [ ]:
#Add a new character to the existing DataFrame by specifying its characteristics. Restict Duplicates
#This line wont work since it's not ready for authorization
requests.post('http://127.0.0.1:5000/characters', 
              params={'Character ID':32342,
                      'Character Name':'Jesus Solis',
                      'Total Available Events':27,
                      'Total Available Series': 3,
                      'Total Available Comics': 32,
                      'Price of the Most Expensive Comic' : 32.99}).json()

**6. To add a new character to the existing DataFrame, specify its ID and the API will retrieve the rest from the marvel API.**

**Duplicates will not work**

In [ ]:
#Add a new character to the existing DataFrame by specifying its ID and retrieve the rest from marvel API. Restict Duplicates
#This line wont work since it's not ready for authorization
requests.post('http://127.0.0.1:5000/characters', params={'Character ID':1009151}).json()

**7. To delete a character from the DataFrame, specify its ID**

In [ ]:
#Delete a character or a list of characters by providing either the Character ID or the Character Nam
#This line wont work since it's not ready for authorization
requests.delete('http://127.0.0.1:5000/characters',params={'Character ID':1009151}).json()

**8. This piece of code, saves a csv file to your computer with the Email and Password needed to maniuplate the API**

In [ ]:
#Create the sign in form
df = pd.DataFrame(columns=['email', 'password'])
df.to_csv('users.csv', index=False)

**9. This code posts the values for email and password and changes the saved User csv with new credentials.**

In [ ]:
# Sign up module
requests.post('http://127.0.0.1:5000/signup', params={'email': 'jesus.solis@esade.edu', 'password': '1303'}).json()

**10. This part provides the User with the access token needed for the API. This token is then saved so the user doesn't have to input the token in every piece of code.**

In [ ]:
#log In module
response = requests.get('http://127.0.0.1:5000/login', params={'email': 'jesus.solis@esade.edu', 'password': '1303'}).json()
access_token = response['token']
access_token

**11. This section enables the user to add a new character to the existing DataFrame by specifying its characteristics, alter the values for the distinict keys in the params dictionary, by using your authorization token.**

**Duplicates will not work**

In [ ]:
# Post a new character given all the parameters
#This line will work since we've passed authorization token 
requests.post('http://127.0.0.1:5000/characters', 
              params={'Character ID':32342,
                      'Character Name':'Jesus Solis',
                      'Total Available Events':27,
                      'Total Available Series': 3,
                      'Total Available Comics': 32,
                      'Price of the Most Expensive Comic' : 32.99},
              headers={'Authorization': f'Bearer {access_token}'}).json()

**12. In this part we will test our code and add characters in different ways altering our approach an the information we provide when posting.** 

In [ ]:
# Post a new character only with the character id in marvels page
requests.post('http://127.0.0.1:5000/characters', 
              params={'Character ID':1011361},
              headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
# Post a character that doesn't exist in marvels API
requests.post('http://127.0.0.1:5000/characters', 
              params={'Character ID':32302},
              headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
#This line will work since we've passed authorization token 
requests.delete('http://127.0.0.1:5000/characters'
                ,
                headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
#This line will work since we've passed authorization token 
requests.delete('http://127.0.0.1:5000/characters'
                ,params={'Character ID':32342},
                headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
#This line will work since we've passed authorization token 
requests.delete('http://127.0.0.1:5000/characters'
                ,params={'Character ID':[1009146,1011361]},
                headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
#This line will work since we've passed authorization token 
requests.delete('http://127.0.0.1:5000/characters'
                ,params={'Character Name':'Albion'},
                headers={'Authorization': f'Bearer {access_token}'}).json()

## Bonus: Updating most expensive comic price
A characters most expensive comic price can be updated with a variety of symbols, which will be converted from given `currency` to `usd`.

Forex rates are fetched with a [3rd-party API](https://apilayer.com/marketplace/exchangerates_data-api) that supports all available currencies

Either the `id` or `name` parameter has to be passed in order to identify the entry. If both are passed, 400 Bad Request will be returned due to the conflicting query.

Accepted parameters:
```
id: id of the character, required if name is null
name: name of the character, required if id is null
price: The new price of the characters most expensive comic in given currency - REQUIRED
currency: The ISO 4217 Code of the currency that price is specified in - REQUIRED
```

In [ ]:
import requests
requests.put('http://127.0.0.1:5000/characters', params={'Character ID':1011334,
                                                         'Price': 4,
                                                        'Currency': "chf"}).json()
